# Features in Active Table:

1. UserId - Id of the user who is interacting with the content

2. postId - Id of the post the viewers interact with

3. Category - category of the post(eg. Art,Food,Animal,Nature)

4. eventType - [0-8] = 
                        0 - view(below 30%)
                        1 - Rewatch
                        2 - seen(30% to 60%)
                        3 - spentTime(60%-80%)
                        4 - fully watched(80% - 100%)
                        5 - like
                        6 - share
                        7 - comment
                        8 - bookmark(save story)
                        
5. event_Date - date of the 

6. post_owned_by(id)

7. posted_datetime - the time and date of posting the content

8. Account_type - (public/private)

9. ignore_post - [0-1] = 
                        0 - User post
                        1 - creator shorts

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date

In [7]:
#importing the interaction table

In [8]:
df = pd.read_csv('User_Interaction.csv')

In [9]:
df.columns

Index(['timestamp', 'eventType', 'contentId', 'personId', 'sessionId',
       'userAgent', 'userRegion', 'userCountry'],
      dtype='object')

In [10]:
#import the post table

In [11]:
df2=pd.read_csv('article.csv')

In [12]:
len(df2.contentId.unique())

3057

In [13]:
df2.lang.unique()

array(['en', 'pt', 'es', 'la', 'ja'], dtype=object)

In [14]:
articles_df = df2
articles_df = pd.read_csv('article.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
#removing unwanted columns
articles_df = articles_df.drop(articles_df.columns[[ 1,5,6,7,8]], axis=1)

In [15]:
articles_df.head(1)

,timestamp,contentId,authorPersonId,authorSessionId,url,title,text,lang
1,1459193988,-4.110354e+18,4340306774493623681,8940341205206233829,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en


# Features used

**personId** - UserId

**contentId** - postId

**lang(substitute)** - Category

**eventType** - eventType(0-5)

**timestamp(user_interaction)** - event_Date

**authorPersonId** - post_owned_by

**timestamp(artcile)** - posted_date

**None** - Account_Type

**None** - ignore_post

In [16]:
articles_df = articles_df.rename(columns = {'timestamp':'posted_date','contentId':'postId','authorPersonId':'post_owned_by','lang':'Category'})
df= df.rename(columns = {'personId':'UserId','contentId':'postId','timestamp':'event_Date'})
df = df.drop(df.columns[[5,6,7]], axis=1)
articles_df = articles_df.drop(articles_df.columns[[4,5,6]], axis=1)
articles_df = articles_df.drop(articles_df.columns[[3]], axis=1)
articles_df.head(1)


,posted_date,postId,post_owned_by,Category
1,1459193988,-4.110354e+18,4340306774493623681,en


In [17]:
# rewatch can be used as acriteria to retain in active table

In [18]:
#merge both the tables

In [19]:
new_df = df.merge(articles_df)

In [20]:
len(new_df.postId.unique())

2897

In [21]:
new_df = new_df.drop(new_df.columns[[4]], axis=1)

In [22]:
new_df.head(3)

,event_Date,eventType,postId,UserId,posted_date,post_owned_by,Category
0,1465413032,VIEW,-3.499919e+18,-8845298781299428018,1465309434,-1032019229384696495,en
1,1465413046,VIEW,-3.499919e+18,-8845298781299428018,1465309434,-1032019229384696495,en
2,1465907360,VIEW,-3.499919e+18,-108842214936804958,1465309434,-1032019229384696495,en


In [23]:
new_df.Category.value_counts()

en    45098
pt    24326
la       54
ja       34
es        7
Name: Category, dtype: int64

In [24]:
# replacing the string label with a integer

VIEW - 0

LIKE - 1

BOOKMARK - 2

COMMENT CREATED - 3

FOLLOW - 4

In [25]:
new_df['eventType']=new_df['eventType'].replace(['VIEW'], 0)
new_df['eventType']=new_df['eventType'].replace(['LIKE'], 1)
new_df['eventType']=new_df['eventType'].replace(['BOOKMARK'], 2)
new_df['eventType']=new_df['eventType'].replace(['COMMENT CREATED'], 3)
new_df['eventType']=new_df['eventType'].replace(['FOLLOW'], 4)

In [26]:
new_df.eventType.value_counts()

0    58718
1     5529
2     2381
3     1556
4     1335
Name: eventType, dtype: int64

In [27]:
new_df.head(5)

,event_Date,eventType,postId,UserId,posted_date,post_owned_by,Category
0,1465413032,0,-3.499919e+18,-8845298781299428018,1465309434,-1032019229384696495,en
1,1465413046,0,-3.499919e+18,-8845298781299428018,1465309434,-1032019229384696495,en
2,1465907360,0,-3.499919e+18,-108842214936804958,1465309434,-1032019229384696495,en
3,1465583412,0,-3.499919e+18,-1443636648652872475,1465309434,-1032019229384696495,en
4,1465582468,0,-3.499919e+18,-1443636648652872475,1465309434,-1032019229384696495,en


In [28]:
#storing the new dataframe in active table dataframe

In [29]:
active_table = new_df

In [30]:
#changing the timestamp to date and time format

In [31]:
active_table['event_Date']=pd.to_datetime(active_table['event_Date'],unit='s')

In [32]:
active_table['posted_date']=pd.to_datetime(active_table['posted_date'],unit='s')

In [33]:
active_table['postId']=active_table.postId.astype(str)

In [34]:
len(active_table['event_Date'].unique())

66044

In [35]:
active_table['event_Only_Date']=active_table['event_Date'].dt.date
active_table['event_Only_Date'] = active_table['event_Only_Date'].astype('datetime64[ns]')
active_table['posted_Only_Date']=active_table['posted_date'].dt.date
active_table['posted_Only_Date'] = active_table['posted_Only_Date'].astype('datetime64[ns]')

In [36]:
custom_date = pd.DataFrame({'custom_date': pd.date_range('2017-02-28', periods=1)})

In [37]:
active_table['new_date'] = pd.to_datetime(custom_date['custom_date'])
active_table['new_date']=active_table['new_date'].dt.date
active_table['new_date'] = active_table['new_date'].astype('datetime64[ns]')

In [38]:
active_table['new_date'].fillna('2017-02-28', inplace = True)

In [39]:
active_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69519 entries, 0 to 69518
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   event_Date        69519 non-null  datetime64[ns]
 1   eventType         69519 non-null  int64         
 2   postId            69519 non-null  object        
 3   UserId            69519 non-null  int64         
 4   posted_date       69519 non-null  datetime64[ns]
 5   post_owned_by     69519 non-null  int64         
 6   Category          69519 non-null  object        
 7   event_Only_Date   69519 non-null  datetime64[ns]
 8   posted_Only_Date  69519 non-null  datetime64[ns]
 9   new_date          69519 non-null  datetime64[ns]
dtypes: datetime64[ns](5), int64(3), object(2)
memory usage: 5.8+ MB


In [40]:
len(active_table.postId.unique())

2897

In [41]:
active_table['Interval(btw_posted_and_event)']=active_table['event_Only_Date'].sub(active_table['posted_Only_Date'], axis=0)
active_table['Interval(btw_recorded_and_event)']=active_table['new_date'].sub(active_table['event_Only_Date'], axis=0)
active_table['Interval(btw_recorded_and_posted)']=active_table['new_date'].sub(active_table['posted_Only_Date'], axis=0)

In [42]:
active_table.head(2)

,event_Date,eventType,postId,UserId,posted_date,post_owned_by,Category,event_Only_Date,posted_Only_Date,new_date,Interval(btw_posted_and_event),Interval(btw_recorded_and_event),Interval(btw_recorded_and_posted)
0,2016-06-08 19:10:32,0,-3.499919498720039e+18,-8845298781299428018,2016-06-07 14:23:54,-1032019229384696495,en,2016-06-08,2016-06-07,2017-02-28,1 days,265 days,266 days
1,2016-06-08 19:10:46,0,-3.499919498720039e+18,-8845298781299428018,2016-06-07 14:23:54,-1032019229384696495,en,2016-06-08,2016-06-07,2017-02-28,1 days,265 days,266 days


#### Remove rows with wrong date

In [43]:
improper_date=active_table[active_table['posted_date'] > active_table['event_Date']]

In [44]:
active_table=active_table.drop(improper_date.index)

In [45]:
active_table.head()

,event_Date,eventType,postId,UserId,posted_date,post_owned_by,Category,event_Only_Date,posted_Only_Date,new_date,Interval(btw_posted_and_event),Interval(btw_recorded_and_event),Interval(btw_recorded_and_posted)
0,2016-06-08 19:10:32,0,-3.499919498720039e+18,-8845298781299428018,2016-06-07 14:23:54,-1032019229384696495,en,2016-06-08,2016-06-07,2017-02-28,1 days,265 days,266 days
1,2016-06-08 19:10:46,0,-3.499919498720039e+18,-8845298781299428018,2016-06-07 14:23:54,-1032019229384696495,en,2016-06-08,2016-06-07,2017-02-28,1 days,265 days,266 days
2,2016-06-14 12:29:20,0,-3.499919498720039e+18,-108842214936804958,2016-06-07 14:23:54,-1032019229384696495,en,2016-06-14,2016-06-07,2017-02-28,7 days,259 days,266 days
3,2016-06-10 18:30:12,0,-3.499919498720039e+18,-1443636648652872475,2016-06-07 14:23:54,-1032019229384696495,en,2016-06-10,2016-06-07,2017-02-28,3 days,263 days,266 days
4,2016-06-10 18:14:28,0,-3.499919498720039e+18,-1443636648652872475,2016-06-07 14:23:54,-1032019229384696495,en,2016-06-10,2016-06-07,2017-02-28,3 days,263 days,266 days


In [46]:
len(active_table.postId.unique())

2893

In [47]:
#create a table called history table which will be empty at first

In [48]:
history_table = active_table.drop(active_table.index)

In [49]:
history_table

,event_Date,eventType,postId,UserId,posted_date,post_owned_by,Category,event_Only_Date,posted_Only_Date,new_date,Interval(btw_posted_and_event),Interval(btw_recorded_and_event),Interval(btw_recorded_and_posted)


In [50]:
#Post with highest number of interactions

In [51]:
eventLike=active_table[active_table['eventType']==1]
active_table['eventLike']=eventLike['eventType']
count_like = active_table.groupby(['postId','eventLike']).count().groupby(['postId']).max()
count_like_sorted=count_like.sort_values(by='eventType',ascending=False).head(633)
count_like_sorted

,event_Date,eventType,UserId,posted_date,post_owned_by,Category,event_Only_Date,posted_Only_Date,new_date,Interval(btw_posted_and_event),Interval(btw_recorded_and_event),Interval(btw_recorded_and_posted)
postId,,,,,,,,,,,,
8.224860111193159e+18,38,38,38,38,38,38,38,38,38,38,38,38
-1.633984990770981e+18,35,35,35,35,35,35,35,35,35,35,35,35
-6.783772548752092e+18,31,31,31,31,31,31,31,31,31,31,31,31
9.438180269308984e+17,30,30,30,30,30,30,30,30,30,30,30,30
-2.358756719610362e+18,28,28,28,28,28,28,28,28,28,28,28,28
...,...,...,...,...,...,...,...,...,...,...,...,...
2.0486575522510607e+18,3,3,3,3,3,3,3,3,3,3,3,3
7.985545205318798e+18,3,3,3,3,3,3,3,3,3,3,3,3
7.521700660190109e+18,3,3,3,3,3,3,3,3,3,3,3,3


### Filtering the posts without any views

In [52]:
view = active_table[active_table['eventType']==0].iloc[:,2]
like = active_table[active_table['eventType']==1].iloc[:,2]
bookmark = active_table[active_table['eventType']==2].iloc[:,2]
comment = active_table[active_table['eventType']==3].iloc[:,2]
follow = active_table[active_table['eventType']==4].iloc[:,2]

view = pd.DataFrame(view)
like = pd.DataFrame(like)
bookmark = pd.DataFrame(bookmark)
comment = pd.DataFrame(comment)
follow = pd.DataFrame(follow)

follow=follow.drop_duplicates()
view=view.drop_duplicates()
like=like.drop_duplicates()
comment=comment.drop_duplicates()
bookmark=bookmark.drop_duplicates()

follow_view = pd.concat([view,follow])
comment_view = pd.concat([view,comment])
like_view = pd.concat([view,like])
bookmark_view = pd.concat([view,bookmark])

follow_view=follow_view.drop_duplicates()
like_view=follow_view.drop_duplicates()
comment_view=follow_view.drop_duplicates()
bookmark_view=follow_view.drop_duplicates()

duplicate_follow=pd.concat([follow_view,follow,view]).drop_duplicates()
duplicate_like=pd.concat([like_view,like]).drop_duplicates()
duplicate_bookmark=pd.concat([bookmark_view,bookmark]).drop_duplicates()
duplicate_comment=pd.concat([comment_view,comment]).drop_duplicates()

#Posts with no views
No_Views = ['5.929055844564383e+18','3.8010640607978086e+18','-5.289386635926629e+18','-8.935051432668166e+18','5.232169374213305e+18','-1.737937277055037e+18']

### Remove the posts with no views

In [53]:
active_table = active_table[active_table['postId'] != '5.929055844564383e+18']
active_table = active_table[active_table['postId'] != '3.8010640607978086e+18']
active_table = active_table[active_table['postId'] != '-5.289386635926629e+18']
active_table = active_table[active_table['postId'] != '-8.935051432668166e+18']
active_table = active_table[active_table['postId'] != '5.232169374213305e+18']
active_table = active_table[active_table['postId'] != '-1.737937277055037e+18']

### Number of Unique Posts in the dataset

In [54]:
len(active_table.postId.unique())

2887

### Posts which had "LIKE" interactions in the past 180 days

In [55]:
eventLike=active_table[(active_table['eventType']==1)]

active_table['eventLike']=eventLike['eventType']

eventLike_sorted=eventLike.sort_values(by=['Interval(btw_recorded_and_posted)','Interval(btw_recorded_and_event)'],ascending=[False,False])

eventLike_sorted.reset_index(inplace=True)

eventLike_sorted = eventLike_sorted.drop(eventLike_sorted.columns[[0,1]], axis=1)

eventLike_sorted['Interval(btw_recorded_and_event)']=eventLike_sorted['Interval(btw_recorded_and_event)'].astype(str)
eventLike_sorted['Interval(btw_recorded_and_event)']=eventLike_sorted['Interval(btw_recorded_and_event)'].str.replace(r'[^\d]','', regex=True)
eventLike_sorted['Interval(btw_recorded_and_event)'] = eventLike_sorted['Interval(btw_recorded_and_event)'].astype(int)

like_event=eventLike_sorted['Interval(btw_recorded_and_event)'] < 180

data = []
for i in range(len(like_event)):
    
    if like_event[i] == True:
        event_postId = eventLike_sorted[eventLike_sorted.index == i]
        #event_postId = event_postId['postId']
        data.append(event_postId['postId'])

        
print(len(np.unique(data)))
data

485


[224    3.0689756144325386e+18
 Name: postId, dtype: object,
 383    1.507299850679912e+18
 Name: postId, dtype: object,
 491    3.756874058846e+18
 Name: postId, dtype: object,
 546    -5.3502990529496115e+17
 Name: postId, dtype: object,
 674    -6.843047699859121e+18
 Name: postId, dtype: object,
 675    -6.843047699859121e+18
 Name: postId, dtype: object,
 676    -6.843047699859121e+18
 Name: postId, dtype: object,
 677    -6.843047699859121e+18
 Name: postId, dtype: object,
 996    1.7380525932264215e+18
 Name: postId, dtype: object,
 1306    5.109221095427625e+18
 Name: postId, dtype: object,
 1766    9.110374552591359e+17
 Name: postId, dtype: object,
 1767    9.110374552591359e+17
 Name: postId, dtype: object,
 1887    1.5546190402894152e+18
 Name: postId, dtype: object,
 1978    8.806127404334958e+17
 Name: postId, dtype: object,
 2107    2.555983212310147e+18
 Name: postId, dtype: object,
 2590    8.386493184609385e+18
 Name: postId, dtype: object,
 2591    2.7199092534198026

In [58]:
active_table['eventType'].value_counts()

0    58623
1     5520
2     2375
3     1555
4     1334
Name: eventType, dtype: int64

### Posts which had "LIKE or COMMENT" interactions in the past 180 days

In [56]:
#filter both like and comment
eventLike_comment=active_table[(active_table['eventType']==1) | (active_table['eventType']==3)]
print(eventLike_comment.shape)

active_table['eventLike_comment']=eventLike_comment['eventType']
eventLike_comment_sorted=eventLike_comment.sort_values(by=['Interval(btw_recorded_and_posted)','Interval(btw_recorded_and_event)']
                              ,ascending=[True,True])
eventLike_comment_sorted.reset_index(inplace=True)

eventLike_comment_sorted['Interval(btw_recorded_and_event)']=eventLike_comment_sorted['Interval(btw_recorded_and_event)'].astype(str)
eventLike_comment_sorted['Interval(btw_recorded_and_event)']=eventLike_comment_sorted['Interval(btw_recorded_and_event)'].str.replace(r'[^\d]','', regex=True)
eventLike_comment_sorted['Interval(btw_recorded_and_event)'] = eventLike_comment_sorted['Interval(btw_recorded_and_event)'].astype(int)

comment_like_event=eventLike_comment_sorted['Interval(btw_recorded_and_event)'] < 30

data_comment_like=[]
for i in range(len(comment_like_event)):
    
    if comment_like_event[i] == True:
        event_postId = eventLike_comment_sorted[eventLike_comment_sorted.index == i]
        #event_postId = event_postId['postId']
        data_comment_like.append(event_postId['postId'])
        
print(len(np.unique(data_comment_like)))

data_comment_like

(7075, 14)
45


[0    4.10961889034302e+18
 Name: postId, dtype: object,
 1    -5.953227649059337e+18
 Name: postId, dtype: object,
 2    -4.1323314045536266e+18
 Name: postId, dtype: object,
 3    -4.1323314045536266e+18
 Name: postId, dtype: object,
 4    -4.1323314045536266e+18
 Name: postId, dtype: object,
 5    -4.1323314045536266e+18
 Name: postId, dtype: object,
 6    -4.1323314045536266e+18
 Name: postId, dtype: object,
 7    -4.1323314045536266e+18
 Name: postId, dtype: object,
 8    -4.1323314045536266e+18
 Name: postId, dtype: object,
 9    -6.340141548068598e+18
 Name: postId, dtype: object,
 10    -6.340141548068598e+18
 Name: postId, dtype: object,
 11    -6.340141548068598e+18
 Name: postId, dtype: object,
 12    -6.340141548068598e+18
 Name: postId, dtype: object,
 13    -6.340141548068598e+18
 Name: postId, dtype: object,
 14    -6.872546942144599e+18
 Name: postId, dtype: object,
 15    -6.872546942144599e+18
 Name: postId, dtype: object,
 16    -6.872546942144599e+18
 Name: postId, 

In [59]:
active_table.columns

Index(['event_Date', 'eventType', 'postId', 'UserId', 'posted_date',
       'post_owned_by', 'Category', 'event_Only_Date', 'posted_Only_Date',
       'new_date', 'Interval(btw_posted_and_event)',
       'Interval(btw_recorded_and_event)', 'Interval(btw_recorded_and_posted)',
       'eventLike', 'eventLike_comment'],
      dtype='object')

In [57]:
# Saving a story should have the highest priority 

### Work to be done:

1. Appending the posts which did not have any interaction in the past 180 days to the history data
2. Removing the said data in active table

### Miscellaneous codes

eventView_test=active_table[(active_table['eventType']==0)]
active_table['eventView_test']=eventView_test['eventType']
eventView_test=eventView_test.sort_values(by='Interval(btw_recorded_and_event)',ascending=False)

count_df_date = active_table.groupby(['postId','eventView_test']).count().groupby(['postId']).sum()
count_df_date_sorted=count_df_date.sort_values(by='eventType',ascending=True)

count_like = active_table.groupby(['postId','eventLike']).count().groupby(['postId','eventLike']).sum()
count_like_sorted=count_like.sort_values(by='eventType',ascending=False)
count_like_sorted

active_table['eventView_test']=eventView_test['eventType']
active_table.head()

count_df_date = active_table.groupby(['postId','eventView_test']).max().groupby(['postId','Interval(btw_recorded_and_event)']).sum()
count_df_date = count_df_date.sort_values(by='eventType',ascending=False)
count_df_date

len(active_table.postId.unique())

count_df_date_sorted_dropped=count_df_date_sorted.drop(count_df_date_sorted.columns[[0,3,4,5,6,7,8,9,10,11,12,13,14]], axis=1)

count_df_date_sorted_dropped=count_df_date_sorted_dropped.merge(data,on='postId')

count_df_date_sorted

count_df_date_sorted_dropped=count_df_date_sorted_dropped.rename(columns={'eventType':'viewCount'})

count_df_date_sorted_dropped

count_df_date_sorted_dropped=count_df_date_sorted_dropped.sort_values(by=['Interval(btw_recorded_and_posted)','viewCount'],ascending=[False,True])

count_df_date_sorted_dropped.drop(count_df_date_sorted_dropped.columns[[2]],axis=1,inplace=True)

count_df_date_sorted_dropped

# Appending to history table

cond = active_table.Category == 'en'
rows = active_table.loc[cond, :]
history_table = history_table.append(rows, ignore_index=True)
active_table.drop(rows.index, inplace=False)